In [1]:
#Main Packages
import numpy as np
import pandas as pd
import math
import time
import copy
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import multiprocessing
import random

#Visualizations
import os

#Statistics packages
from scipy.stats import pearsonr 
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve, auc, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

#Others
#import tensorrt

## Import all data

In [2]:
PlayerPos = pd.read_csv('PlayerPos.csv')
Home = pd.read_csv('Home.csv')
Away = pd.read_csv('Away.csv')
VelHome = pd.read_csv('VelHome.csv')
VelAway = pd.read_csv('VelAway.csv')
DiscAccHome = pd.read_csv('DiscAccHome.csv')
DiscAccAway = pd.read_csv('DiscAccAway.csv')
Ball = pd.read_csv('Ball.csv')
VelBall = pd.read_csv('VelBall.csv')
DefRewardList = pd.read_csv('DefRewardListnd3.csv')
AttRewardList = pd.read_csv('AttRewardListnd3.csv')
ValDefRewardList = pd.read_csv('ValDefRewardListnd3.csv')
ValAttRewardList = pd.read_csv('ValAttRewardListnd3.csv')
DefEvaluationList = pd.read_csv('DefEvaluationList.csv')
AttEvaluationList = pd.read_csv('AttEvaluationList.csv')

## Single run code

In [3]:
'''
RewardList = pd.read_csv('RewardList.csv')

RewardList['Reward'] = RewardList['Reward'].shift(-1)

NewRewardList = []
for i in range(1,3):
    for j in range(1,12):
        k = 0
        while k < len(RewardList):
            FirstTouch = False
            if RewardList.iloc[k][3] != j or RewardList.iloc[k][0] == 0 or RewardList.iloc[k][2] != i:
                NewRewardList.append([RewardList.iloc[k][0],RewardList.iloc[k][1],RewardList.iloc[k][2],RewardList.iloc[k][3],i,j,RewardList.iloc[k][4]])
                k += 1
            else:
                if RewardList.iloc[k-1][0] == 0:
                    FirstTouch = True
                while RewardList.iloc[k][3] == j and RewardList.iloc[k][2] == i:
                    k += 1
                if RewardList.iloc[k][0] != 0 and not FirstTouch:
                    NewRewardList.append([0,'Ball Received',i,j,0,0,0])

PlayerRewardList = pd.DataFrame(NewRewardList,columns = ['Match_index', 'Frame','Team','Player','Agent Team','Agent Player','Reward'])

PlayerRewardList.to_csv('PlayerRewardList.csv',header = True, index = False)

RewardList = pd.read_csv('PlayerRewardList.csv')

DefRewardList = RewardList[RewardList['Team'] != RewardList['Agent Team']]

i = 0
while i < len(DefRewardList):
    if i == 0 and DefRewardList['Match_index'].iloc[i] == 0:
        DefRewardList = DefRewardList.drop(DefRewardList.index[i])
    else:
        while DefRewardList['Match_index'].iloc[i] != 0:
            i += 1
        i += 1
        while i < len(DefRewardList) and DefRewardList['Match_index'].iloc[i] == 0:
            DefRewardList = DefRewardList.drop(DefRewardList.index[i])
    

DefRewardList['Reward'] = DefRewardList['Reward'].multiply(-1)

DefRewardList.to_csv('DefRewardList.csv', index = False)

DefRewardList = pd.read_csv('DefRewardList.csv')

AttRewardList = RewardList[(RewardList['Team'] == RewardList['Agent Team']) | (RewardList['Match_index']==0)]

i = 0
while i < len(AttRewardList):
    if i == 0 and AttRewardList['Match_index'].iloc[i] == 0:
        AttRewardList = AttRewardList.drop(AttRewardList.index[i])
    else:
        while AttRewardList['Match_index'].iloc[i] != 0:
            i += 1
        i += 1
        while i < len(AttRewardList) and AttRewardList['Match_index'].iloc[i] == 0:
            AttRewardList = AttRewardList.drop(AttRewardList.index[i])
    

AttRewardList.to_csv('AttRewardList.csv', index = False)

AttRewardList = pd.read_csv('AttRewardList.csv')

def get_cardinal_direction(vector):
    # Define cardinal directions
    directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]

    # Calculate angle and magnitude
    angle = math.atan2(vector[0], vector[1])
    angle_degrees = math.degrees(angle)
    magnitude = math.sqrt(vector[0] ** 2 + vector[1] ** 2)

    # Find the closest cardinal direction
    index = round(angle_degrees / 45) % 8
    closest_direction = directions[index]

    return closest_direction, magnitude
AccHome = pd.read_csv('AccHome.csv')
AccAway = pd.read_csv('AccAway.csv')

i = 0
while i < len(AccHome):
    j = 1
    while j < 12:
        Direction, Magnitude = get_cardinal_direction([AccHome.iloc[i,2*j],AccHome.iloc[i,2*j + 1]])
        AccHome.iloc[i,2*j] = Magnitude
        AccHome.iloc[i,2*j + 1] = Direction
        j += 1
    i += 1


AccHome.to_csv('DiscAccHome.csv', index = False)

i = 0
while i < len(AccAway):
    j = 1
    while j < 12:
        Direction, Magnitude = get_cardinal_direction([AccAway.iloc[i,2*j],AccAway.iloc[i,2*j + 1]])
        AccAway.iloc[i,2*j] = Magnitude
        AccAway.iloc[i,2*j + 1] = Direction
        j += 1
    i += 1


AccAway.to_csv('DiscAccAway.csv', index = False)

#AccAway.describe()

'''

'\nRewardList = pd.read_csv(\'RewardList.csv\')\n\nRewardList[\'Reward\'] = RewardList[\'Reward\'].shift(-1)\n\nNewRewardList = []\nfor i in range(1,3):\n    for j in range(1,12):\n        k = 0\n        while k < len(RewardList):\n            FirstTouch = False\n            if RewardList.iloc[k][3] != j or RewardList.iloc[k][0] == 0 or RewardList.iloc[k][2] != i:\n                NewRewardList.append([RewardList.iloc[k][0],RewardList.iloc[k][1],RewardList.iloc[k][2],RewardList.iloc[k][3],i,j,RewardList.iloc[k][4]])\n                k += 1\n            else:\n                if RewardList.iloc[k-1][0] == 0:\n                    FirstTouch = True\n                while RewardList.iloc[k][3] == j and RewardList.iloc[k][2] == i:\n                    k += 1\n                if RewardList.iloc[k][0] != 0 and not FirstTouch:\n                    NewRewardList.append([0,\'Ball Received\',i,j,0,0,0])\n\nPlayerRewardList = pd.DataFrame(NewRewardList,columns = [\'Match_index\', \'Frame\',\'Team\

In [4]:
'''
## Code for non discounted dataset###
ValAttRewardList = AttRewardList.loc[(AttRewardList['Match_index']==3) | (AttRewardList['Match_index'] == 0)]

i = 0
while i < len(ValAttRewardList):
    if ValAttRewardList.iloc[i][0] == 3:
        i += 1
    else:
        if i != 0:
            i += 1
        while i < len(ValAttRewardList) and ValAttRewardList.iloc[i][0] == 0:
            ValAttRewardList = ValAttRewardList.drop(ValAttRewardList.index[i])
        

AttRewardList = AttRewardList.drop(ValAttRewardList.index)

ValDefRewardList = DefRewardList.loc[(DefRewardList['Match_index']==3) | (DefRewardList['Match_index'] == 0)]

i = 0
while i < len(ValDefRewardList):
    if ValDefRewardList.iloc[i][0] == 3:
        i += 1
    else:
        if i != 0:
            i += 1
        while i < len(ValDefRewardList) and ValDefRewardList.iloc[i][0] == 0:
            ValDefRewardList = ValDefRewardList.drop(ValDefRewardList.index[i])
        

DefRewardList = DefRewardList.drop(ValDefRewardList.index)

AttRewardList.to_csv('AttRewardListnd3.csv', index = False)
DefRewardList.to_csv('DefRewardListnd3.csv', index = False)
ValAttRewardList.to_csv('ValAttRewardListn3.csv', index = False)
ValDefRewardList.to_csv('ValDefRewardListnd3.csv', index = False)
'''

"\n## Code for non discounted dataset###\nValAttRewardList = AttRewardList.loc[(AttRewardList['Match_index']==3) | (AttRewardList['Match_index'] == 0)]\n\ni = 0\nwhile i < len(ValAttRewardList):\n    if ValAttRewardList.iloc[i][0] == 3:\n        i += 1\n    else:\n        if i != 0:\n            i += 1\n        while i < len(ValAttRewardList) and ValAttRewardList.iloc[i][0] == 0:\n            ValAttRewardList = ValAttRewardList.drop(ValAttRewardList.index[i])\n        \n\nAttRewardList = AttRewardList.drop(ValAttRewardList.index)\n\nValDefRewardList = DefRewardList.loc[(DefRewardList['Match_index']==3) | (DefRewardList['Match_index'] == 0)]\n\ni = 0\nwhile i < len(ValDefRewardList):\n    if ValDefRewardList.iloc[i][0] == 3:\n        i += 1\n    else:\n        if i != 0:\n            i += 1\n        while i < len(ValDefRewardList) and ValDefRewardList.iloc[i][0] == 0:\n            ValDefRewardList = ValDefRewardList.drop(ValDefRewardList.index[i])\n        \n\nDefRewardList = DefRewardL

In [5]:
'''
PlayerRewardList.insert(len(PlayerRewardList.columns),'First_state', np.zeros(len(PlayerRewardList)),True)

i = 0
while i < len(PlayerRewardList):
    if i == 0:
        PlayerRewardList.iloc[i,-1] = 1
        i += 1
    else:
        if PlayerRewardList.iloc[i-1,0] == 0:
            PlayerRewardList.iloc[i,-1] = 1
        i += 1

EvaluationList = PlayerRewardList.loc[PlayerRewardList['Match_index'] != 0]
DefEvaluationList = EvaluationList[EvaluationList['Team'] != EvaluationList['Agent Team']].reset_index(drop = True)
AttEvaluationList = EvaluationList[EvaluationList['Team'] == EvaluationList['Agent Team']].reset_index(drop = True)

DefEvaluationList.to_csv('DefEvaluationList.csv',header = True, index = False)
AttEvaluationList.to_csv('AttEvaluationList.csv',header = True, index = False)
'''

"\nPlayerRewardList.insert(len(PlayerRewardList.columns),'First_state', np.zeros(len(PlayerRewardList)),True)\n\ni = 0\nwhile i < len(PlayerRewardList):\n    if i == 0:\n        PlayerRewardList.iloc[i,-1] = 1\n        i += 1\n    else:\n        if PlayerRewardList.iloc[i-1,0] == 0:\n            PlayerRewardList.iloc[i,-1] = 1\n        i += 1\n\nEvaluationList = PlayerRewardList.loc[PlayerRewardList['Match_index'] != 0]\nDefEvaluationList = EvaluationList[EvaluationList['Team'] != EvaluationList['Agent Team']].reset_index(drop = True)\nAttEvaluationList = EvaluationList[EvaluationList['Team'] == EvaluationList['Agent Team']].reset_index(drop = True)\n\nDefEvaluationList.to_csv('DefEvaluationList.csv',header = True, index = False)\nAttEvaluationList.to_csv('AttEvaluationList.csv',header = True, index = False)\n"

# Generator

In [6]:
def FrameToIndex(Match_index, Frame): #A function that receives a frame and outputs the corresponding dataframe index
    return Ball.loc[(Ball['Frame']==int(Frame)) & (Ball['Match_index'] == Match_index)].index[0]

In [7]:
class QLearningDataGenerator(Dataset):
    def __init__(self, reward_list):
        self.reward_list = reward_list
        self.No_zeros = self.reward_list.loc[self.reward_list['Match_index'] != 0]
        self.eval_flag = len(self.reward_list) == len(self.No_zeros)
        # Load dataFrames outside the constructor for efficiency
        #self.PlayerPos = pd.read_csv('PlayerPos.csv')
        #self.Home = pd.read_csv('Home.csv')
        #self.Away = pd.read_csv('Away.csv')
        #self.VelHome = pd.read_csv('VelHome.csv')
        #self.VelAway = pd.read_csv('VelAway.csv')
        #self.AccHome = pd.read_csv('DiscAccHome.csv')
        #self.AccAway = pd.read_csv('DiscAccAway.csv')
        #self.Ball = pd.read_csv('Ball.csv')
        #self.VelBall = pd.read_csv('VelBall.csv')
        
    def __len__(self):
        return len(self.No_zeros)
    def __getitem__(self, idx):
        idx = self.__len__() - idx - 1
        Match_index, Frame, Team, Player, AgentTeam, AgentPlayer, reward = self.No_zeros.iloc[idx]
        original_id = self.No_zeros.index[idx]
        if self.eval_flag:
            final_state = False
        else:
            final_state = self.reward_list.iloc[original_id + 1][0] == 0  # Check next row for episode end
        state = self.get_state(Match_index, Frame, Team, Player, AgentTeam, AgentPlayer, final_state)
        action = self.get_action(Match_index, Frame, Team, Player, AgentTeam, AgentPlayer)
        return state, action, reward

    def get_state(self, Match_index, Frame,Team,Player,AgentTeam,AgentPlayer, final_state):
        AgentPlayer = int(AgentPlayer) #Fixes bug with evaluation dataframe
        n = FrameToIndex(Match_index, Frame)
        MatOpp = np.zeros([60,100])
        MatTeam = np.zeros([60,100])
        MatOppvx = np.zeros([60,100])
        MatOppvy = np.zeros([60,100])
        MatTeamvx = np.zeros([60,100])
        MatTeamvy = np.zeros([60,100])
        MatBall = np.zeros([60,100])
        MatBallvx = np.zeros([60,100])
        MatBallvy = np.zeros([60,100])
        MatAge = np.zeros([60,100])
        MatAgevx = np.zeros([60,100])
        MatAgevy = np.zeros([60,100])
        xball = Ball.iloc[n,2]
        yball = Ball.iloc[n,3]
        MatBall[yball][xball] = 1
        MatBallvx[yball][xball] = VelBall.iloc[n,2]
        MatBallvy[yball][xball] = VelBall.iloc[n,3]
        if AgentTeam==1:
            yopp = Away.iloc[n, [2*i+2 for i in range(1, 12)]].to_numpy()
            xopp = Away.iloc[n, [2*i+1 for i in range(1, 12)]].to_numpy()
            yteam = Home.iloc[n, [2*i+2 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            xteam = Home.iloc[n, [2*i+1 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            yopp = yopp.round().astype(int)  # Round and convert to integer
            xopp = xopp.round().astype(int)
            yteam = yteam.round().astype(int)
            xteam = xteam.round().astype(int)
            xvelteam = VelHome.iloc[n,[2*i for i in range(1,12) if i != AgentPlayer]].to_numpy()
            yvelteam = VelHome.iloc[n,[2*i+1 for i in range(1,12) if i != AgentPlayer]].to_numpy()
            xvelopp = VelAway.iloc[n,[2*i for i in range(1,12)]].to_numpy()
            yvelopp = VelAway.iloc[n,[2*i+1 for i in range(1,12)]].to_numpy()
            MatOpp[yopp, xopp] = 1
            MatOppvx[yopp, xopp] = xvelopp
            MatOppvy[yopp, xopp] = yvelopp
            MatTeam[yteam, xteam] = 1
            MatTeamvx[yteam, xteam] = xvelteam
            MatTeamvy[yteam, xteam] = yvelteam
            yage = Home.iloc[n,2*AgentPlayer + 2]
            xage = Home.iloc[n,2*AgentPlayer + 1]
            xvelage = VelHome.iloc[n,2*AgentPlayer]
            yvelage = VelHome.iloc[n,2*AgentPlayer + 1]
            MatAge[yage, xage] = 1
            MatAgevx[yage, xage] = xvelage
            MatAgevy[yage, xage] = yvelage
        if AgentTeam==2:
            yopp = Home.iloc[n, [2*i+2 for i in range(1, 12)]].to_numpy()
            xopp = Home.iloc[n, [2*i+1 for i in range(1, 12)]].to_numpy()
            yteam = Away.iloc[n, [2*i+2 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            xteam = Away.iloc[n, [2*i+1 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            yopp = yopp.round().astype(int)  # Round and convert to integer
            xopp = xopp.round().astype(int)
            yteam = yteam.round().astype(int)
            xteam = xteam.round().astype(int)
            xvelteam = VelAway.iloc[n,[2*i for i in range(1,12) if i != AgentPlayer]].to_numpy()
            yvelteam = VelAway.iloc[n,[2*i+1 for i in range(1,12) if i != AgentPlayer]].to_numpy()
            xvelopp = VelHome.iloc[n,[2*i for i in range(1,12)]].to_numpy()
            yvelopp = VelHome.iloc[n,[2*i+1 for i in range(1,12)]].to_numpy()
            MatOpp[yopp, xopp] = 1
            MatOppvx[yopp, xopp] = xvelopp
            MatOppvy[yopp, xopp] = yvelopp
            MatTeam[yteam, xteam] = 1
            MatTeamvx[yteam, xteam] = xvelteam
            MatTeamvy[yteam, xteam] = yvelteam
            yage = Away.iloc[n,2*AgentPlayer + 2]
            xage = Away.iloc[n,2*AgentPlayer + 1]
            xvelage = VelAway.iloc[n,2*AgentPlayer]
            yvelage = VelAway.iloc[n,2*AgentPlayer + 1]
            MatAge[yage, xage] = 1
            MatAgevx[yage, xage] = xvelage
            MatAgevy[yage, xage] = yvelage
        MatOpp = torch.from_numpy(MatOpp)
        MatTeam = torch.from_numpy(MatTeam)
        MatOppvx = torch.from_numpy(MatOppvx)
        MatOppvy = torch.from_numpy(MatOppvy)
        MatTeamvx = torch.from_numpy(MatTeamvx)
        MatTeamvy = torch.from_numpy(MatTeamvy)
        MatBall = torch.from_numpy(MatBall)
        MatBallvx = torch.from_numpy(MatBallvx)
        MatBallvy = torch.from_numpy(MatBallvy)
        MatAge = torch.from_numpy(MatAge)
        MatAgevx = torch.from_numpy(MatAgevx)
        MatAgevy = torch.from_numpy(MatAgevy)
        return [torch.stack([MatOpp,MatTeam,MatOppvx,MatOppvy,MatTeamvx,MatTeamvy,MatBall,MatBallvx,MatBallvy,MatAge,MatAgevx,MatAgevy]), final_state]


  # Add a method to get action if applicable for your Q-learning approach
    def get_action(self, Match_index, Frame, Team,Player,AgentTeam,AgentPlayer):
        AgentPlayer = int(AgentPlayer)
        n = FrameToIndex(Match_index, Frame)
        if AgentTeam==1:
            Magnitude = DiscAccHome.iloc[n,2*AgentPlayer]
            Direction = DiscAccHome.iloc[n,2*AgentPlayer+1]
        if AgentTeam==2:
            Magnitude = DiscAccAway.iloc[n,2*AgentPlayer]
            Direction = DiscAccAway.iloc[n,2*AgentPlayer+1]
        if Magnitude < 0.2:
            return 'No Acceleration'
        elif Magnitude < 0.5:
            return 'Small, '+Direction
        elif Magnitude < 1:
            return 'Medium, '+Direction
        else:
            return 'Big, '+Direction
        return (Magnitude,Direction)

In [8]:
def_generator = QLearningDataGenerator(DefRewardList)

In [9]:
att_generator = QLearningDataGenerator(AttRewardList)

In [10]:
val_def_generator = QLearningDataGenerator(ValDefRewardList)

In [11]:
val_att_generator = QLearningDataGenerator(ValAttRewardList)

In [12]:
eval_def_generator = QLearningDataGenerator(DefEvaluationList.drop(['First_state'],axis = 1))

In [13]:
eval_att_generator = QLearningDataGenerator(AttEvaluationList.drop(['First_state'],axis = 1))

In [14]:
def_dataloader = DataLoader(def_generator, batch_size=1, shuffle=False, num_workers = 6)

In [15]:
att_dataloader = DataLoader(att_generator, batch_size=1, shuffle=False, num_workers = 6)

In [16]:
val_def_dataloader = DataLoader(val_def_generator, batch_size=1, shuffle=False, num_workers = 6)

In [17]:
val_att_dataloader = DataLoader(val_att_generator, batch_size=1, shuffle=False, num_workers = 6)

In [18]:
eval_def_dataloader = DataLoader(eval_def_generator, batch_size=1, shuffle=False, num_workers = 6)

In [19]:
eval_att_dataloader = DataLoader(eval_att_generator, batch_size=1, shuffle=False, num_workers = 6)

In [20]:
'''
for i in range(1,13):
    att_dataloader = DataLoader(att_generator, batch_size=1, shuffle=False, num_workers = i)
    j = 0
    start_time = time.time()
    for states, actions, rewards in att_dataloader:
        j+=1
        if j == 1000000:
            break
    print('For ',13-i,'workers, it takes ', time.time()-start_time, 'seconds')
'''

"\nfor i in range(1,13):\n    att_dataloader = DataLoader(att_generator, batch_size=1, shuffle=False, num_workers = i)\n    j = 0\n    start_time = time.time()\n    for states, actions, rewards in att_dataloader:\n        j+=1\n        if j == 1000000:\n            break\n    print('For ',13-i,'workers, it takes ', time.time()-start_time, 'seconds')\n"

For  1 workers, it takes  244.71065592765808 seconds

For  2 workers, it takes  139.8216598033905 seconds

For  3 workers, it takes  105.91393327713013 seconds

For  4 workers, it takes  89.58012199401855 seconds

For  5 workers, it takes  82.76936197280884 seconds

For  6 workers, it takes  79.70292806625366 seconds

For  7 workers, it takes  80.15350341796875 seconds

For  8 workers, it takes  81.66234469413757 seconds

For  9 workers, it takes  82.12781977653503 seconds

For  10 workers, it takes  83.10064005851746 seconds

For  11 workers, it takes  83.56236934661865 seconds

## Neural Network

In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional layers with max pooling and ReLU activation
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3)
        #self.norm1 = nn.BatchNorm2d(12)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=2)
        #self.norm2 = nn.BatchNorm2d(12)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()
        #self.conv3 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3)
        #self.norm3 = nn.BatchNorm2d(12)
        #self.pool3 = nn.MaxPool2d(kernel_size=2)
        #self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

        # Flatten the output of convolutional layers
        #self.flatten = torch.flatten()

        # Hidden layers with sigmoid activation
        self.fc1 = nn.Linear(12*14*24, 256)
        #self.fc1 = nn.Linear(12*6*11, 256)
        self.sigmoid = nn.Sigmoid() 
        #self.fc2 = nn.Linear(256, 256)
        #self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 25)

    def forward(self, x):
        # Pass through convolutional layers
        x = self.conv1(x)
        #x = self.norm1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        #x = self.norm2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        #x = self.conv3(x)
        #x = self.norm3(x)
        #x = self.pool3(x)
        #x = self.relu3(x)
        x = self.dropout(x)

        # Flatten the output
        #x = self.flatten(x)
        x = torch.flatten(x)

        # Replace LSTM part with this for simpler model (without sequence processing)
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.dropout(x)
        #x = self.fc2(x)
        #x = self.sigmoid(x)
        #x = self.dropout(x)
        #x = self.fc3(x)
        #x = self.sigmoid(x)
        x = self.fc4(x)

        return x

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
model = CNN().to(device)

In [24]:
model.load_state_dict(torch.load('CurBest.pt'))
model.eval()

CNN(
  (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(12, 12, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=4032, out_features=256, bias=True)
  (sigmoid): Sigmoid()
  (fc4): Linear(in_features=256, out_features=25, bias=True)
)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0000001)

In [26]:
#model(att_generator.__getitem__(0)[0][0].float().to(device))

In [27]:
def ActionToNumber(Action):
    if len(Action) != 1: #For the shuffled case and evaluation
        if Action[0] == 'N': #No Acceleration
            return 0
        elif Action[0] == 'S': #Small Acceleration
            mult = 0
        elif Action[0] == 'M': #Medium Acceleration
            mult = 1
        else: #Big Acceleration
            mult = 2
        Cardinal = Action[-2] + Action[-1]
        Cardinal_list = [' N', 'NE', ' E', 'SE', ' S', 'SW', ' W', 'NW']
        add = Cardinal_list.index(Cardinal) + 1
        return 8 * mult + add
    if Action[0][0] == 'N': #No Acceleration
        return 0
    elif Action[0][0] == 'S': #Small Acceleration
        mult = 0
    elif Action[0][0] == 'M': #Medium Acceleration
        mult = 1
    else: #Big Acceleration
        mult = 2
    Cardinal = Action[0][-2] + Action[0][-1]
    Cardinal_list = [' N', 'NE', ' E', 'SE', ' S', 'SW', ' W', 'NW']
    add = Cardinal_list.index(Cardinal) + 1
    return 8 * mult + add

In [28]:
def GetAdjacentActions(Action):
    AdjacentActions = []
    Cardinal_list = [' N', 'NE', ' E', 'SE', ' S', 'SW', ' W', 'NW']
    if Action[0] == 'N': #No Acceleration
        AdjacentActions += [0]
        IntensityList = ['Small,']
        for i in IntensityList:
            for j in Cardinal_list:
                AdjacentActions += [ActionToNumber(i+j)]
        return AdjacentActions
    elif Action[0] == 'S': #Small Acceleration
        AdjacentActions += [0]
        IntensityList = ['Small,', 'Medium,']
    elif Action[0] == 'M': #Medium Acceleration
        IntensityList = ['Small,', 'Medium,', 'Big,']
    else:
        IntensityList = ['Medium,', 'Big,']
    Cardinal = Action[-2] + Action[-1]
    idx = Cardinal_list.index(Cardinal)
    if idx == 7:
        AdjacentCardinals = [' W', 'NW', ' N']
    else:
        AdjacentCardinals = [Cardinal_list[idx-1], Cardinal_list[idx], Cardinal_list[idx+1]]
    for i in IntensityList:
        for j in AdjacentCardinals:
            AdjacentActions += [ActionToNumber(i+j)]
    return AdjacentActions

In [29]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output, reward, action, next_state, next_action, final_state):
        action_index = ActionToNumber(action)
        next_action_index = ActionToNumber(next_action)
        Qvalue = output[action_index]
        if not final_state:
            output = model(next_state[0].float().to(device))
            AdjacentActions = GetAdjacentActions(action[0])
            OptionsValue = [output[j] for j in AdjacentActions]
            next_Qvalue = max(OptionsValue)
        else:
            next_Qvalue = 0
        reward = reward.to(device)
        loss = (0.955*next_Qvalue + reward - Qvalue)**2
        return loss

In [30]:
loss_fn = CustomLoss()

In [31]:
def shufflesampletrain(epochs):
    size = 10000
    samplevalidation(0)
    lst = list(range(size))
    for j in range(epochs):
        train_loss = 0
        model.train()
        k = 0
        random.shuffle(lst)
        for i in lst:
            cur_state = att_generator.__getitem__(i)[0]
            cur_action = att_generator.__getitem__(i)[1]
            cur_reward = torch.tensor(att_generator.__getitem__(i)[2])
            next_state = att_generator.__getitem__(i+1)[0]
            next_action = att_generator.__getitem__(i+1)[1]
            data = cur_state[0].float().to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
            with torch.no_grad():
                train_loss += loss
            loss.backward()
            optimizer.step()
            k += 1
            prog = format((k/size) * 100, ".2f")
            print('Epoch',j,'progress:' + prog +'%',end='\r')
            if k == size:
                break
        with torch.no_grad():
            train_loss /= size
            print('\nTrain loss is',train_loss)
        samplevalidation(j)

In [32]:
def samplevalidation(j):
    val_size = 3000
    model.eval() 
    val_loss = 0
    with torch.no_grad():
        i = 0
        for state, action, reward in val_def_dataloader:
            if i == 0:
                next_state = state
                next_action = action
                next_reward = reward
                i += 1
            else:
                cur_state = state
                cur_action = action
                cur_reward = reward
                data = cur_state[0].float().to(device)
                output = model(data)
                val_loss += loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                next_state = cur_state
                next_action = cur_action
                next_reward = cur_reward
                i += 1
                prog = format((i/val_size) * 100, ".2f")
                print('Validation',j,'progress:' + prog +'%',end='\r')
                if i == val_size:
                    break
        val_loss /= val_size
        print('\nValidation loss is',val_loss)
        print('')

In [33]:
def sampletrain(epochs):
    size = 10000
    samplevalidation(0)
    for j in range(epochs):
        train_loss = 0
        model.train()
        i = 0
        for state, action, reward in def_dataloader:
            if i == 0:
                next_state = state
                next_action = action
                next_reward = reward
                i += 1
            else:
                cur_state = state
                cur_action = action
                cur_reward = reward
                data = cur_state[0].float().to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                with torch.no_grad():
                    train_loss += loss
                loss.backward()
                optimizer.step()
                next_state = cur_state
                next_action = cur_action
                next_reward = cur_reward
                i += 1
                prog = format((i/size) * 100, ".2f")
                print('Epoch',j,'progress:' + prog +'%',end='\r')
                if i == size:
                    break
        with torch.no_grad():
            train_loss /= size
            print('\nTrain loss is',train_loss)
        samplevalidation(j)

In [34]:
def validation(j):
    val_size = len(val_att_dataloader)
    model.eval() 
    val_loss = 0
    with torch.no_grad():
        i = 0
        for state, action, reward in val_att_dataloader:
            if i == 0:
                next_state = state
                next_action = action
                next_reward = reward
                i += 1
            else:
                cur_state = state
                cur_action = action
                cur_reward = reward
                data = cur_state[0].float().to(device)
                output = model(data)
                val_loss += loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                next_state = cur_state
                next_action = cur_action
                next_reward = cur_reward
                i += 1
                prog = format((i/val_size) * 100, ".2f")
                print('Validation',j,'progress:' + prog +'%',end='\r')
        val_loss /= val_size
        return val_loss

In [35]:
def train(epochs):
    size = len(att_dataloader)
    best_val_loss = validation(0)
    print('\nValidation loss is',best_val_loss)
    print('')
    counter = 0
    for j in range(epochs):
        train_loss = 0
        model.train()
        i = 0
        for state, action, reward in att_dataloader:
            if i == 0:
                next_state = state
                next_action = action
                next_reward = reward
                i += 1
            else:
                cur_state = state
                cur_action = action
                cur_reward = reward
                data = cur_state[0].float().to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                with torch.no_grad():
                    train_loss += loss
                loss.backward()
                optimizer.step()
                next_state = cur_state
                next_action = cur_action
                next_reward = cur_reward
                i += 1
                prog = format((i/size) * 100, ".2f")
                print('Epoch',j,'progress:' + prog +'%',end='\r')
        with torch.no_grad():
            train_loss /= size
            print('\nTrain loss is',train_loss)
        val_loss = validation(j)
        print('\nValidation loss is',val_loss)
        print('')
        if val_loss > best_val_loss:
            counter += 1
            if counter == 10:
                break
            torch.save(model.state_dict(), 'Cur.pt')
        else:
            counter = 0
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'CurBest.pt')
        

In [ ]:
train(100)

Validation 0 progress:100.00%
Validation loss is tensor([0.2412], device='cuda:0', dtype=torch.float64)

Epoch 0 progress:100.00%
Train loss is tensor([0.2311], device='cuda:0', dtype=torch.float64)
Validation 0 progress:100.00%
Validation loss is tensor([0.2410], device='cuda:0', dtype=torch.float64)

Epoch 1 progress:100.00%
Train loss is tensor([0.2310], device='cuda:0', dtype=torch.float64)
Validation 1 progress:100.00%
Validation loss is tensor([0.2408], device='cuda:0', dtype=torch.float64)



1

In [37]:
#train(0)

### New stuff, training on the def model

### Actual training

0.000001 (256,25) 12 channels, dropout 0.3 - 0.1554

0.0000001 (256,25) 12 channels, dropout 0.3 - 



###### Validation 3000

0.001 (256,25) 12 channels, dropout 0.3 - 0.1746

0.001 (256,256,25) 12 channels, dropout 0.3 - 0.1796

0.0001 (256,25) 12 channels - bad




#### Actual training results

0.000001 (256,25) 12 channels, dropout - 0.2269

mixed (0.00001, 0.000001) (256,25) 12 channels, dropout 0.3 - 0.2190



##### Tests with 1000 validation

0.0001 (256,25) 12 channels, dropout 0.1 - 0.1053

0.0001 (256,25) 12 channels, dropout 0.2 - 0.1039

#### Tests with 3000 validation

0.0001 (256,25) 12 channels, dropout 0.2 - 0.2143

0.001 (256,25) 12 channels, dropout 0.2 - 0.2154

0.0001 (256,25) 12 channels, dropout 0.3 - 0.2104

0.0001 (256,256, 25) 12 channels, dropout 0.3 - 0.2182

0.00001 (512,256,25) 12 channels - 0.1232

0.000001 (512,256,25) 12 channels - 0.1218

0.000001 (256,25) 12 channels, dropout - 0.1201

Sample Train:

0.000001 (256,25) 12 channels - 0.0949

0.001 (256,25) 12 channels, dropout - 0.0939

In [38]:
#torch.save(model.state_dict(), 'AttModel.pt')

In [39]:
model(def_generator.__getitem__(0)[0][0].float().to(device))

tensor([-0.0894,  0.1270,  0.1234, -0.1124, -0.0215, -0.0171, -0.0048, -0.1009,
         0.1446,  0.3908,  0.1229, -0.1076, -0.0274,  0.2363, -0.0141, -0.0985,
         0.1271,  0.1250,  0.1414, -0.1046, -0.0280, -0.0226, -0.0129, -0.1109,
         0.1336], device='cuda:0', grad_fn=<ViewBackward0>)

In [40]:
PerformanceList = []
length = len(DefEvaluationList)
with torch.no_grad():
    for Match in range(1,8):
        for Team in range(1,3):
            for Player in range(1,12):
                impact = 0
                Evaldf = DefEvaluationList[(DefEvaluationList['Match_index'] == Match) & (DefEvaluationList['Agent Team'] == Team) & (DefEvaluationList['Agent Player'] == Player)]
                for i in Evaldf.index:
                    idx = length - i - 1
                    state, action, _ = eval_def_generator.__getitem__(idx)
                    if Evaldf.loc[i][-1]:
                        output = model(eval_def_generator.__getitem__(idx)[0][0].float().to(device))
                        ChoiceValue = output[ActionToNumber(action)]
                        impact += ChoiceValue - (sum(output)/len(output))
                    else:
                        _, prev_action, _ = eval_def_generator.__getitem__(idx+1)
                        AdjacentActions = GetAdjacentActions(prev_action)
                        output = model(eval_def_generator.__getitem__(idx)[0][0].float().to(device))
                        OptionsValue = [output[j] for j in AdjacentActions]
                        ChoiceValue = output[ActionToNumber(action)]
                        impact += ChoiceValue - (sum(OptionsValue)/len(OptionsValue))
                impact /= len(Evaldf)
                PerformanceList += [[Match,Team,Player,impact]]
                print(Match,Team,Player,'Done')


1 1 1 Done
1 1 2 Done
1 1 3 Done
1 1 4 Done
1 1 5 Done
1 1 6 Done
1 1 7 Done
1 1 8 Done
1 1 9 Done
1 1 10 Done
1 1 11 Done
1 2 1 Done
1 2 2 Done
1 2 3 Done
1 2 4 Done
1 2 5 Done
1 2 6 Done
1 2 7 Done
1 2 8 Done
1 2 9 Done
1 2 10 Done
1 2 11 Done
2 1 1 Done
2 1 2 Done
2 1 3 Done
2 1 4 Done
2 1 5 Done
2 1 6 Done
2 1 7 Done
2 1 8 Done
2 1 9 Done
2 1 10 Done
2 1 11 Done
2 2 1 Done
2 2 2 Done
2 2 3 Done
2 2 4 Done
2 2 5 Done
2 2 6 Done
2 2 7 Done
2 2 8 Done
2 2 9 Done
2 2 10 Done
2 2 11 Done
3 1 1 Done
3 1 2 Done
3 1 3 Done
3 1 4 Done
3 1 5 Done
3 1 6 Done
3 1 7 Done
3 1 8 Done
3 1 9 Done
3 1 10 Done
3 1 11 Done
3 2 1 Done
3 2 2 Done
3 2 3 Done
3 2 4 Done
3 2 5 Done
3 2 6 Done
3 2 7 Done
3 2 8 Done
3 2 9 Done
3 2 10 Done
3 2 11 Done
4 1 1 Done
4 1 2 Done
4 1 3 Done
4 1 4 Done
4 1 5 Done
4 1 6 Done
4 1 7 Done
4 1 8 Done
4 1 9 Done
4 1 10 Done
4 1 11 Done
4 2 1 Done
4 2 2 Done
4 2 3 Done
4 2 4 Done
4 2 5 Done
4 2 6 Done
4 2 7 Done
4 2 8 Done
4 2 9 Done
4 2 10 Done
4 2 11 Done
5 1 1 Done
5 1 2

In [41]:
PerformanceDataframe = pd.DataFrame(PerformanceList)

In [44]:
i = 0
while i < len(PerformanceDataframe):
    PerformanceDataframe.iloc[i,-1]  = PerformanceDataframe.iloc[i,-1].item()
    i += 1

In [46]:
PerformanceDataframe.columns = ['Match','Team', 'Player', 'Impact']

In [47]:
PerformanceDataframe.to_csv('PerformanceDataframeDef.csv',header = True, index = False)

In [48]:
PerformanceDataframe

,Match,Team,Player,Impact
0,1,1,1,-0.019326
1,1,1,2,-0.024337
2,1,1,3,-0.017073
3,1,1,4,-0.013593
4,1,1,5,-0.024657
...,...,...,...,...
149,7,2,7,-0.024802
150,7,2,8,-0.017199
151,7,2,9,-0.029799
152,7,2,10,-0.020608


In [53]:
U15Dataframe = PerformanceDataframe[PerformanceDataframe['Match'] <= 3]
U17Dataframe = PerformanceDataframe[(PerformanceDataframe['Match'] <= 5) & (PerformanceDataframe['Match'] >= 4)]
U19Dataframe = PerformanceDataframe[PerformanceDataframe['Match'] >= 6]

In [56]:
Dataframe1 = PerformanceDataframe[PerformanceDataframe['Match'] == 1]
Dataframe2 = PerformanceDataframe[PerformanceDataframe['Match'] == 2]
Dataframe3 = PerformanceDataframe[PerformanceDataframe['Match'] == 3]
Dataframe4 = PerformanceDataframe[PerformanceDataframe['Match'] == 4]
Dataframe5 = PerformanceDataframe[PerformanceDataframe['Match'] == 5]
Dataframe6 = PerformanceDataframe[PerformanceDataframe['Match'] == 6]
Dataframe7 = PerformanceDataframe[PerformanceDataframe['Match'] == 7]
Score1 = Dataframe1.sum()[-1]/len(Dataframe1)
Score2 = Dataframe2.sum()[-1]/len(Dataframe2)
Score3 = Dataframe3.sum()[-1]/len(Dataframe3)
Score4 = Dataframe4.sum()[-1]/len(Dataframe4)
Score5 = Dataframe5.sum()[-1]/len(Dataframe5)
Score6 = Dataframe6.sum()[-1]/len(Dataframe6)
Score7 = Dataframe7.sum()[-1]/len(Dataframe7)
print('Game 1 score is:',Score1)
print('Game 2 score is:',Score2)
print('Game 3 score is:',Score3)
print('Game 4 score is:',Score4)
print('Game 5 score is:',Score5)
print('Game 6 score is:',Score6)
print('Game 7 score is:',Score7)

Game 1 score is: -0.025483633137562058
Game 2 score is: -0.022453765173188665
Game 3 score is: -0.028096942612054674
Game 4 score is: -0.03575908155603842
Game 5 score is: -0.03646235582842068
Game 6 score is: -0.02555742889473384
Game 7 score is: -0.018068839838220316


In [54]:
U15Score = U15Dataframe.sum()[-1]/len(U15Dataframe)
U17Score = U17Dataframe.sum()[-1]/len(U17Dataframe)
U19Score = U19Dataframe.sum()[-1]/len(U19Dataframe)
print('U15 score is:',U15Score)
print('U17 score is:',U17Score)
print('U19 score is:',U19Score)

U15 score is: -0.0253447803076018
U17 score is: -0.03611071869222955
U19 score is: -0.02181313436647708


In [66]:
def GetTeamScores(Dataframe):
    Team1Score = 0
    Team2Score = 0
    for i in range(len(Dataframe)):
        if Dataframe.iloc[i][1] == 1:
            Team1Score += Dataframe.iloc[i][-1]
        else:
            Team2Score += Dataframe.iloc[i][-1]
    print(Team1Score,Team2Score)

In [68]:
GetTeamScores(Dataframe1)
GetTeamScores(Dataframe2)
GetTeamScores(Dataframe3)
GetTeamScores(Dataframe4)
GetTeamScores(Dataframe5)
GetTeamScores(Dataframe6)
GetTeamScores(Dataframe7)

-0.2810705006122589 -0.27956942841410637
-0.25771714374423027 -0.23626569006592035
-0.33122526854276657 -0.28690746892243624
-0.41126235388219357 -0.37543744035065174
-0.3867097236216068 -0.4154621046036482
-0.29826067201793194 -0.26400276366621256
-0.17087524756789207 -0.22663922887295485


### Scores:

Home 0 - 1 Away

Home 8 - 1 Away

Home 4 - 3 Away

Home 7 - 2 Away

Home 2 - 0 Away

Home 1 - 0 Away

Home 4 - 3 Away

Code to save

In [ ]:
class QLearningDataGenerator(Dataset):
    def __init__(self, reward_list):
        self.reward_list = reward_list
        # Load dataFrames outside the constructor for efficiency
        #self.PlayerPos = pd.read_csv('PlayerPos.csv')
        #self.Home = pd.read_csv('Home.csv')
        #self.Away = pd.read_csv('Away.csv')
        #self.VelHome = pd.read_csv('VelHome.csv')
        #self.VelAway = pd.read_csv('VelAway.csv')
        #self.AccHome = pd.read_csv('DiscAccHome.csv')
        #self.AccAway = pd.read_csv('DiscAccAway.csv')
        #self.Ball = pd.read_csv('Ball.csv')
        #self.VelBall = pd.read_csv('VelBall.csv')
        
    def __len__(self):
        return len(self.reward_list.loc[self.reward_list['Match_index'] != 0])
    def __getitem__(self, idx):
        No_zeros = self.reward_list.loc[self.reward_list['Match_index'] != 0]
        Match_index, Frame, Team, Player, AgentTeam, AgentPlayer, reward = No_zeros.iloc[idx]
        original_id = No_zeros.index[idx]
        final_state = self.reward_list.iloc[original_id + 1][0] == 0  # Check next row for episode end
        state = self.get_state(Match_index, Frame, Team, Player, AgentTeam, AgentPlayer, final_state)
        action = self.get_action(Match_index, Frame, Team, Player, AgentTeam, AgentPlayer)
        return state, action, reward

    def get_state(self, Match_index, Frame,Team,Player,AgentTeam,AgentPlayer, final_state):
        n = FrameToIndex(Match_index, Frame)
        MatOpp = np.zeros([60,100])
        MatTeam = np.zeros([60,100])
        MatOppvx = np.zeros([60,100])
        MatOppvy = np.zeros([60,100])
        MatTeamvx = np.zeros([60,100])
        MatTeamvy = np.zeros([60,100])
        MatBall = np.zeros([60,100])
        MatBallvx = np.zeros([60,100])
        MatBallvy = np.zeros([60,100])
        MatAge = np.zeros([60,100])
        MatAgevx = np.zeros([60,100])
        MatAgevy = np.zeros([60,100])
        xball = Ball.iloc[n,2]
        yball = Ball.iloc[n,3]
        MatBall[yball][xball] = 1
        MatBallvx[yball][xball] = VelBall.iloc[n,2]
        MatBallvy[yball][xball] = VelBall.iloc[n,3]
        if AgentTeam==1:
            yopp = Away.iloc[n, [2*i+2 for i in range(1, 12)]].to_numpy()
            xopp = Away.iloc[n, [2*i+1 for i in range(1, 12)]].to_numpy()
            yteam = Home.iloc[n, [2*i+2 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            xteam = Home.iloc[n, [2*i+1 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            yopp = yopp.round().astype(int)  # Round and convert to integer
            xopp = xopp.round().astype(int)
            yteam = yteam.round().astype(int)
            xteam = xteam.round().astype(int)
            xvelteam = VelHome.iloc[n,[2*i for i in range(1,12) if i != AgentPlayer]].to_numpy()
            yvelteam = VelHome.iloc[n,[2*i+1 for i in range(1,12) if i != AgentPlayer]].to_numpy()
            xvelopp = VelAway.iloc[n,[2*i for i in range(1,12)]].to_numpy()
            yvelopp = VelAway.iloc[n,[2*i+1 for i in range(1,12)]].to_numpy()
            MatOpp[yopp, xopp] = 1
            MatOppvx[yopp, xopp] = xvelopp
            MatOppvy[yopp, xopp] = yvelopp
            MatTeam[yteam, xteam] = 1
            MatTeamvx[yteam, xteam] = xvelteam
            MatTeamvy[yteam, xteam] = yvelteam
            yage = Home.iloc[n,2*AgentPlayer + 2]
            xage = Home.iloc[n,2*AgentPlayer + 1]
            xvelage = VelHome.iloc[n,2*AgentPlayer]
            yvelage = VelHome.iloc[n,2*AgentPlayer + 1]
            MatAge[yage, xage] = 1
            MatAgevx[yage, xage] = xvelage
            MatAgevy[yage, xage] = yvelage
        if AgentTeam==2:
            yopp = Home.iloc[n, [2*i+2 for i in range(1, 12)]].to_numpy()
            xopp = Home.iloc[n, [2*i+1 for i in range(1, 12)]].to_numpy()
            yteam = Away.iloc[n, [2*i+2 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            xteam = Away.iloc[n, [2*i+1 for i in range(1, 12) if i != AgentPlayer]].to_numpy()
            yopp = yopp.round().astype(int)  # Round and convert to integer
            xopp = xopp.round().astype(int)
            yteam = yteam.round().astype(int)
            xteam = xteam.round().astype(int)
            xvelteam = VelAway.iloc[n,[2*i for i in range(1,12) if i != AgentPlayer]].to_numpy()
            yvelteam = VelAway.iloc[n,[2*i+1 for i in range(1,12) if i != AgentPlayer]].to_numpy()
            xvelopp = VelHome.iloc[n,[2*i for i in range(1,12)]].to_numpy()
            yvelopp = VelHome.iloc[n,[2*i+1 for i in range(1,12)]].to_numpy()
            MatOpp[yopp, xopp] = 1
            MatOppvx[yopp, xopp] = xvelopp
            MatOppvy[yopp, xopp] = yvelopp
            MatTeam[yteam, xteam] = 1
            MatTeamvx[yteam, xteam] = xvelteam
            MatTeamvy[yteam, xteam] = yvelteam
            yage = Away.iloc[n,2*AgentPlayer + 2]
            xage = Away.iloc[n,2*AgentPlayer + 1]
            xvelage = VelAway.iloc[n,2*AgentPlayer]
            yvelage = VelAway.iloc[n,2*AgentPlayer + 1]
            MatAge[yage, xage] = 1
            MatAgevx[yage, xage] = xvelage
            MatAgevy[yage, xage] = yvelage
        MatOpp = torch.from_numpy(MatOpp)
        MatTeam = torch.from_numpy(MatTeam)
        MatOppvx = torch.from_numpy(MatOppvx)
        MatOppvy = torch.from_numpy(MatOppvy)
        MatTeamvx = torch.from_numpy(MatTeamvx)
        MatTeamvy = torch.from_numpy(MatTeamvy)
        MatBall = torch.from_numpy(MatBall)
        MatBallvx = torch.from_numpy(MatBallvx)
        MatBallvy = torch.from_numpy(MatBallvy)
        MatAge = torch.from_numpy(MatAge)
        MatAgevx = torch.from_numpy(MatAgevx)
        MatAgevy = torch.from_numpy(MatAgevy)
        return [torch.stack([MatOpp,MatTeam,MatOppvx,MatOppvy,MatTeamvx,MatTeamvy,MatBall,MatBallvx,MatBallvy,MatAge,MatAgevx,MatAgevy]), final_state]


  # Add a method to get action if applicable for your Q-learning approach
    def get_action(self, Match_index, Frame, Team,Player,AgentTeam,AgentPlayer):
        n = FrameToIndex(Match_index, Frame)
        if AgentTeam==1:
            Magnitude = DiscAccHome.iloc[n,2*AgentPlayer]
            Direction = DiscAccHome.iloc[n,2*AgentPlayer+1]
        if AgentTeam==2:
            Magnitude = DiscAccAway.iloc[n,2*AgentPlayer]
            Direction = DiscAccAway.iloc[n,2*AgentPlayer+1]
        if Magnitude < 0.2:
            return 'No Acceleration'
        elif Magnitude < 0.5:
            return 'Small, '+Direction
        elif Magnitude < 1:
            return 'Medium, '+Direction
        else:
            return 'Big, '+Direction
        return (Magnitude,Direction)

In [ ]:
def train(epochs):
    size = len(att_dataloader)
    #validation(0)
    for j in range(epochs):
        train_loss = 0
        model.train()
        i = 0
        for state, action, reward in att_dataloader:
            if i == 0:
                cur_state = state
                cur_action = action
                cur_reward = reward
                i += 1
            else:
                next_state = state
                next_action = action
                next_reward = reward
                data = cur_state[0].float().to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                with torch.no_grad():
                    train_loss += loss
                    print(train_loss)
                loss.backward()
                optimizer.step()
                cur_state = next_state
                cur_action = next_action
                cur_reward = next_reward
                i += 1
                prog = format((i/size) * 100, ".2f")
                print('Epoch',j,'progress:' + prog +'%',end='\r')
        with torch.no_grad():
            train_loss /= size
            print('\nTrain loss is',train_loss)
        model.eval() 
        validation(j)

In [ ]:
def validation(j):
    val_size = len(val_att_dataloader)
    model.eval() 
    val_loss = 0
    print('')
    with torch.no_grad():
        i = 0
        for state, action, reward in val_att_dataloader:
            if i == 0:
                cur_state = state
                cur_action = action
                cur_reward = reward
                i += 1
            else:
                next_state = state
                next_action = action
                next_reward = reward
                data = cur_state[0].float().to(device)
                output = model(data)
                val_loss += loss_fn(output,cur_reward,cur_action,next_state,next_action,cur_state[1])
                cur_state = next_state
                cur_action = next_action
                cur_reward = next_reward
                i += 1
                prog = format((i/val_size) * 100, ".2f")
                print('Validation',j,'progress:' + prog +'%',end='\r')
        val_loss /= val_size
        print('\nValidation loss is',val_loss)

In [ ]:
PerformanceList = []
length = len(DefEvaluationList)
with torch.no_grad():
    for Match in range(1,8):
        for Team in range(1,3):
            for Player in range(1,12):
                impact = 0
                Evaldf = DefEvaluationList[(DefEvaluationList['Match_index'] == Match) & (DefEvaluationList['Agent Team'] == Team) & (DefEvaluationList['Agent Player'] == Player)]
                for i in Evaldf.index:
                    idx = length - i - 1
                    state, action, _ = eval_def_generator.__getitem__(idx)
                    if Evaldf.loc[i][-1]:
                        output = model(eval_def_generator.__getitem__(idx)[0][0].float().to(device))
                        ChoiceValue = output[ActionToNumber(action)]
                        impact += ChoiceValue - (sum(output)/len(output))
                    else:
                        _, prev_action, _ = eval_def_generator.__getitem__(idx+1)
                        AdjacentActions = GetAdjacentActions(prev_action)
                        output = model(eval_def_generator.__getitem__(idx)[0][0].float().to(device))
                        OptionsValue = [output[j] for j in AdjacentActions]
                        ChoiceValue = output[ActionToNumber(action)]
                        impact += ChoiceValue - (sum(OptionsValue)/len(OptionsValue))
                impact /= len(Evaldf)
                PerformanceList += [[Match,Team,Player,impact]]
                print(Match,Team,Player,'Done')
